# Patient Data Query

This notebook queries patient data from multiple databases for a given prontuario.

## Tables Queried:
- **Clinisys Silver Layer** (clinisys_all.duckdb):
  - view_micromanipulacao
  - view_micromanipulacao_oocitos
  - view_tratamentos
  - view_embrioes_congelados
- **Finops Silver Layer** (huntington_data_lake.duckdb):
  - silver.mesclada_vendas


In [51]:
import duckdb
import pandas as pd
import os
from datetime import datetime

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## Input: Patient Prontuario


In [52]:
def get_clinisys_connection():
    """Get connection to clinisys_all database"""
    clinisys_db_path = os.path.join('..', '..', 'database', 'clinisys_all.duckdb')
    return duckdb.connect(clinisys_db_path, read_only=True)

def get_huntington_connection():
    """Get connection to huntington_data_lake database"""
    huntington_db_path = os.path.join('..', '..', 'database', 'huntington_data_lake.duckdb')
    return duckdb.connect(huntington_db_path, read_only=True)


## Database Connections


In [53]:
def query_micromanipulacao(conn, prontuario):
    """Query view_micromanipulacao for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO ===")
    query = f"""
    SELECT * FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    ORDER BY codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_micromanipulacao")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_micromanipulacao_oocitos(conn, prontuario):
    """Query view_micromanipulacao_oocitos for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO_OOCITOS ===")
    
    # First get codigo_ficha from view_micromanipulacao for this prontuario
    codigos_query = f"""
    SELECT codigo_ficha FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    """
    
    codigos_df = conn.execute(codigos_query).df()
    if not codigos_df.empty:
        codigo_list = codigos_df['codigo_ficha'].tolist()
        placeholders = ','.join(['?'] * len(codigo_list))
        
        oocitos_query = f"""
        SELECT * FROM silver.view_micromanipulacao_oocitos
        WHERE id_micromanipulacao IN ({placeholders})
        ORDER BY id_micromanipulacao, id
        """
        
        df = conn.execute(oocitos_query, codigo_list).df()
        print(f"Found {len(df)} records in view_micromanipulacao_oocitos")
        if not df.empty:
            display(df)
        else:
            print("No records found.")
    else:
        print("No codigo_ficha found for this prontuario in view_micromanipulacao.")
        df = pd.DataFrame()
    
    return df


In [54]:
def query_biopsy_pgta_timeline(conn, prontuario):
    """Query biopsy and PGT-A timeline data for a specific prontuario"""
    print(f"\n=== BIOPSY PGT-A TIMELINE ===")
    query = f"""
    SELECT * FROM gold.biopsy_pgta_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in biopsy_pgta_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_billing_timeline(conn, prontuario):
    """Query billing timeline data for a specific prontuario"""
    print(f"\n=== BILLING TIMELINE ===")
    query = f"""
    SELECT * FROM gold.billing_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in billing_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df


In [55]:
def query_tratamentos(conn, prontuario):
    """Query view_tratamentos for a specific prontuario"""
    print(f"\n=== VIEW_TRATAMENTOS ===")
    query = f"""
    SELECT * FROM silver.view_tratamentos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_tratamentos")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_embrioes_congelados(conn, prontuario):
    """Query view_embrioes_congelados for a specific prontuario"""
    print(f"\n=== VIEW_EMBRIOES_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_embrioes_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_embrioes_congelados")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_mesclada_vendas(conn, prontuario):
    """Query silver.mesclada_vendas for a specific prontuario"""
    print(f"\n=== SILVER.MESCLADA_VENDAS ===")
    query = f"""
    SELECT * FROM silver.mesclada_vendas 
    WHERE prontuario = {prontuario}
    ORDER BY "DT Emissao" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.mesclada_vendas")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df


In [56]:
def query_comprehensive_timeline(conn, prontuario):
    """Query comprehensive timeline data for a specific prontuario"""
    print(f"\n=== COMPREHENSIVE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.comprehensive_biopsy_pgta_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in comprehensive_biopsy_pgta_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df


In [57]:
def query_resumed_timeline(conn, prontuario):
    """Query resumed timeline data for a specific prontuario (most recent record only)"""
    print(f"\n=== RESUMED TIMELINE (MOST RECENT RECORD) ===")
    query = f"""
    SELECT * FROM gold.resumed_biopsy_pgta_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_biopsy_pgta_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df


In [58]:
# Updated query function with biopsy PGT-A timeline
def query_patient_data_with_timeline(prontuario):
    """
    Query all patient data including timeline data from gold layer tables.
    
    Args:
        prontuario (str or int): The patient prontuario number
        
    Returns:
        dict: Dictionary containing all DataFrames with results including timeline data
    """
    print(f"\n{'='*80}")
    print(f"QUERYING DATA FOR PRONTUARIO: {prontuario}")
    print(f"Query executed at: {datetime.now()}")
    print(f"{'='*80}")
    
    # Connect to databases
    clinisys_con = get_clinisys_connection()
    huntington_con = get_huntington_connection()
    
    try:
        # Query clinisys tables
        df_micromanipulacao = query_micromanipulacao(clinisys_con, prontuario)
        df_oocitos = query_micromanipulacao_oocitos(clinisys_con, prontuario)
        df_tratamentos = query_tratamentos(clinisys_con, prontuario)
        df_embrioes = query_embrioes_congelados(clinisys_con, prontuario)
        
        # Query finops table
        df_mesclada = query_mesclada_vendas(huntington_con, prontuario)
        
        # Query biopsy PGT-A timeline
        df_biopsy_pgta = query_biopsy_pgta_timeline(huntington_con, prontuario)
        
        # Query billing timeline
        df_billing = query_billing_timeline(huntington_con, prontuario)
        
        # Query comprehensive timeline
        df_comprehensive = query_comprehensive_timeline(huntington_con, prontuario)
        
        # Query resumed timeline (most recent record only)
        df_resumed = query_resumed_timeline(huntington_con, prontuario)
        
        # Summary
        print(f"\n{'='*80}")
        print(f"QUERY SUMMARY FOR PRONTUARIO {prontuario}")
        print(f"{'='*80}")
        print(f"- view_micromanipulacao: {len(df_micromanipulacao)} records")
        print(f"- view_micromanipulacao_oocitos: {len(df_oocitos)} records")
        print(f"- view_tratamentos: {len(df_tratamentos)} records")
        print(f"- view_embrioes_congelados: {len(df_embrioes)} records")
        print(f"- silver.mesclada_vendas: {len(df_mesclada)} records")
        print(f"- biopsy_pgta_timeline: {len(df_biopsy_pgta)} records")
        print(f"- billing_timeline: {len(df_billing)} records")
        print(f"- comprehensive_timeline: {len(df_comprehensive)} records")
        print(f"- resumed_timeline: {len(df_resumed)} records")
        print(f"{'='*80}")
        
        # Return results as dictionary
        results = {
            'micromanipulacao': df_micromanipulacao,
            'micromanipulacao_oocitos': df_oocitos,
            'tratamentos': df_tratamentos,
            'embrioes_congelados': df_embrioes,
            'mesclada_vendas': df_mesclada,
            'biopsy_pgta_timeline': df_biopsy_pgta,
            'billing_timeline': df_billing,
            'comprehensive_timeline': df_comprehensive,
            'resumed_timeline': df_resumed
        }
        
        return results
        
    finally:
        # Close connections
        clinisys_con.close()
        huntington_con.close()
        print("\nDatabase connections closed.")


## Query Finops Silver Layer Table


In [67]:
# Example usage with comprehensive timeline data
# prontuario = '825_890'  # Patient with PGT-A data
prontuario = '749_579' # 36 biopsies (3 paid) - 28 pgta (16 paid)
# prontuario = '150_059' # 25 biopsies (3 paid) - 19 pgta (0 paid)

# prontuario = '158_696' # 40 biopsies (10 paid) - 17 pgta (6 paid)
# prontuario = '896_947' # 20 biopsies (0 paid) - 20 pgta (0 paid)
# prontuario = '882_954' # 21 biopsies (1 paid) - 21 pgta (3 paid)
# prontuario = '879_286' # 17 biopsies (1 paid) - 17 pgta (4 paid)
# prontuario = '809_017' # 22 biopsies (7 paid) - 22 pgta (4 paid)
# prontuario = '825_263' # 19 biopsies (1 paid) - 15 pgta (0 paid)
# prontuario = '511_257' # 30 biopsies (25 paid) - 25 pgta (8 paid)
# prontuario = '221_379' # 22 biopsies (1 paid) - 10 pgta (0 paid)
# prontuario = '839_259' # 21 biopsies (10 paid) - 10 pgta (10 paid)
# prontuario = '794_640' # 16 biopsies (13 paid) - 13 pgta (6 paid)
# prontuario = '872_534' # 23 biopsies (4 paid) - 6 pgta (6 paid)
# prontuario = '666_290' # 22 biopsies (8 paid) - 19 pgta (8 paid)
# prontuario = '153_411' # 15 biopsies (8 paid) - 13 pgta (8 paid)
# prontuario = '813_209' # 17 biopsies (15 paid) - 11 pgta (19 paid)
# prontuario = '826_457' # 18 biopsies (1 paid) - 18 pgta (21 paid)
# prontuario = '835_937' # 35 biopsies (21 paid) - 35 pgta (25 paid)
# prontuario = '851_077' # 17 biopsies (2 paid) - 17 pgta (10 paid)
# prontuario = '804_527' # 16 biopsies (1 paid) - 16 pgta (5 paid)
# prontuario = '877_692' # 16 biopsies (10 paid) - 10 pgta (5 paid)
# prontuario = '884_227' # 13 biopsies (0 paid) - 13 pgta (0 paid)
# prontuario = '868_915' # 13 biopsies (13 paid) - 13 pgta (4 paid)
# prontuario = '897_679' # 16 biopsies (0 paid) - 9 pgta (0 paid)
# prontuario = '813_112' # 24 biopsies (1 paid) - 2 pgta (0 paid)
# prontuario = '792_259' # 19 biopsies (6 paid) - 18 pgta (6 paid)
# prontuario = '183_278' # 12 biopsies (2 paid) - 12 pgta (2 paid)
# prontuario = '832_016' # 15 biopsies (2 paid) - 15 pgta (15 paid)
# prontuario = '667_294' # 12 biopsies (1 paid) - 12 pgta (5 paid)
# prontuario = '830_663' # 15 biopsies (1 paid) - 15 pgta (5 paid)

# prontuario = '111240'


# Query all patient data including comprehensive timeline
results_with_timeline = query_patient_data_with_timeline(prontuario)

# Access comprehensive timeline data
comprehensive_timeline = results_with_timeline['comprehensive_timeline']
if not comprehensive_timeline.empty:
    print(f"\nComprehensive Timeline for prontuario {prontuario}:")
    print(f"Total biopsies: {comprehensive_timeline['biopsy_count'].sum()}")
    print(f"Total biopsy payments: {comprehensive_timeline['biopsy_payment_count'].sum()}")
    print(f"Biopsy missing payments: {comprehensive_timeline['biopsy_missing_payment'].iloc[0]}")
    print(f"Total PGT-A tests: {comprehensive_timeline['pgta_test_count'].sum()}")
    print(f"Total PGT-A payments: {comprehensive_timeline['pgta_payment_count'].sum()}")
    print(f"PGT-A missing payments: {comprehensive_timeline['pgta_missing_payment'].iloc[0]}")
else:
    print(f"No comprehensive timeline data found for prontuario {prontuario}")



QUERYING DATA FOR PRONTUARIO: 749_579
Query executed at: 2025-10-02 16:28:49.522790

=== VIEW_MICROMANIPULACAO ===
Found 4 records in view_micromanipulacao


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,codigo_congelamento_semen,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,abstinencia_recomendada,recepcao_ovulos,CicloDoadora,IdadeDoadora,checagem_d4,Denud_DL,checagem_fertilizacao,checagem_d2,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,sangue_cateter_interno,dificuldade_transf,retorno_embrioes,vezes_retorno,sangue_cateter_externo,ICSIDescongelados,CicloDescongelamento,pgd_oocito,pgd_d3,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,mini_gradiente,data_procedimento_2,tecnica_injetados_2,motilidade_2,morfologia_injetados_2,unidade_id,data_microtese,data_microtese_2,validade7,responsavel_denud_id,profissional_fertilizacao_id,profissional_d2_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,validade11,materiais_insumos,local_coleta,fonte_outros,gradiente_descontinuo_2,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,mini_gradiente_2,local_coleta_2,fonte_outros_2,observacoes_resumo_ciclos,viscosidade_2,volume_amostra_utilizada_2,morfologia_pre_2,morfologia_pos_2,zymot_2,validade20,gradiente,lote21,validade21,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,c_ps1,d_ps1,conc_ps1,diametro_minimo_pse,diametro_medio_pse,diametro_maximo_pse,tubulos_avaliados_pse,peso_total_pse,peso_microtese_pse,peso_biopsia_pse,diametro_minimo_psd,diametro_medio_psd,diametro_maximo_psd,tubulos_avaliados_psd,peso_total_psd,peso_microtese_psd,peso_biopsia_psd,responsavel_transfer_id,MorfologiaAlterada,EstadoSptz,CongelamentoSemenHom,VialsDescongeladasHom,CongelamentoSemenHet,VialsDescongeladasHet,check_gradiente_ps,concentrado,check_swimup,check_lavagem,volume_ps2,MotivoCongelamentoOvulos,concentracao_progressivos_pos,c_ps2,d_ps2,conc_ps2,comentarios_ps,responsavel_ps,fonte2,inicio_ps2,termino_ps2,volume_ps3,concentracao_progressivos_pre_2,c_ps3,d_ps3,conc_ps3,diametro_minimo_pse2,diametro_medio_pse2,diametro_maximo_pse2,tubulos_avaliados_pse2,peso_total_pse2,peso_microtese_pse2,peso_biopsia_pse2,diametro_minimo_psd2,diametro_medio_psd2,diametro_maximo_psd2,tubulos_avaliados_psd2,peso_total_psd2,peso_microtese_psd2,peso_biopsia_psd2,timelapse,EstadoSptz2,codigo_congelamento_semen_2,CongelamentoSemenHom2,VialsDescongeladasHom2,CongelamentoSemenHet2,VialsDescongeladasHet2,check_gradiente_ps2,concentrado_2,check_swimup2,check_lavagem2,volume_ps4,concentracao_progressivos_pos_2,c_ps4,d_ps4,conc_ps4,comentarios_ps2,responsavel_ps2,transferidos,cateter,lote_cateter,validade_cateter,motivo_nao_transferencia,Transfer_D5,medico_responsavel_id,azoospermia,cod1_crio,cod2_crio,tanque,cane,tecnica,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,responsavel_biopsia,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,22081,2877/24,749579,37,30,2024-08-29,506,11:50,13:20,07:55,21,5293,None,Não,None,None,None,11:10,09:15,None,12:25,07:20,08:30,None,24,43,None,None,None,None,None,Não,None,1,None,1,2024-08-29,ICSI,39 espermatozoides móveis progressivos\r\n,39 espermatozoides normais\r\n,None,1,None,None,None,None,0000000005,<NA>,<NA>,<NA>,5293,5293,None,5293,5293,


=== VIEW_MICROMANIPULACAO_OOCITOS ===
Found 100 records in view_micromanipulacao_oocitos


,id,id_micromanipulacao,diaseguinte,Maturidade,RC,ComentariosAntes,Embriologista,PI,TCD,AH,PGD,ResultadoPGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,GD1,OocitoDoado,ICSI,OrigemOocito,InseminacaoOocito,NClivou_D2,NCelulas_D2,Frag_D2,Blastomero_D2,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,GD2,NClivou_D4,NCelulas_D4,Compactando_D4,MassaInterna_D4,Trofoblasto_D4,NClivou_D5,NCelulas_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,NCelulas_D6,Compactando_D6,MassaInterna_D6,Trofoblasto_D6,NClivou_D7,NCelulas_D7,Compactando_D7,MassaInterna_D7,Trofoblasto_D7,score_maia,relatorio_ia,hash,extraction_timestamp,flag_embryoscope,embryo_number
0,161913,17223,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,None,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,dc7a753f46ae419a68de50a7b6b44887,2025-07-21 21:40:02,0,<NA>
1,161914,17223,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,None,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,376386bc906768c87d5cd7430ba35dc4,2025-07-21 21:40:02,0,<NA>
2,161915,17223,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,None,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,005c5d45efbad107587aca5803399539,2025-07-21 21:40:02,0,<NA>
3,161916,17223,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,None,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,154ca58d96589ad2f826fcaf9a88648c,2025-07-21 21:40:02,0,<NA>
4,161917,17223,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,None,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0e8c6748baf45b19bbc45a5c7d6e289e,2025-07-21 21:40:02,0,<NA>
5,161918,17223,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,None,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,9255d881110c34a756e0b04ae4b58d08,2025-07-21 21:40:02,0,<NA>
6,161919,17223,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,None,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,8387f55283834dac1af74944fa578b5a,2025-07-21 21:40:02,0,<NA>
7,161920,17223,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,None,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,39c8f05cdd32e37a80ad7ab7ade33150,2025-07-21 21:40:02,0,<NA>
8,161921,17223,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,None,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0095274fbfdddb0e9054b2aa272dda56,2025-07-21 21:40:02,0,<NA>
9,161922,17223,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,None,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,39f51a89a5a94b0da300e6bcf3d04c91,2025-07-21 2


=== VIEW_TRATAMENTOS ===
Found 8 records in view_tratamentos


...


=== COMPREHENSIVE TIMELINE ===
Found 7 records in comprehensive_biopsy_pgta_timeline


,prontuario,period_month,biopsy_count,cumulative_biopsy_count,biopsy_payment_amount,biopsy_payment_count,biopsy_payment_qtd,cumulative_biopsy_payment_qtd,biopsy_missing_payment,pgta_test_count,cumulative_pgta_test_count,pgta_payment_amount,pgta_payment_count,pgta_payment_qtd,cumulative_pgta_payment_qtd,pgta_missing_payment
0,749579,2024-10,0,36.0,0.00,0,0.0,3.0,33.0,0,28.0,2369.09,1,2.0,16.0,12.0
1,749579,2024-09,0,36.0,0.00,0,0.0,3.0,33.0,0,28.0,6186.04,1,5.0,14.0,14.0
2,749579,2024-08,24,36.0,2834.11,1,1.0,3.0,33.0,22,28.0,0.00,0,0.0,9.0,19.0
3,749579,2024-05,6,12.0,2806.26,1,1.0,2.0,10.0,6,6.0,5864.10,1,5.0,9.0,-3.0
4,749579,2023-11,0,6.0,0.00,0,0.0,1.0,5.0,0,0.0,4400.00,1,4.0,4.0,-4.0
5,749579,2023-10,6,6.0,0.00,0,0.0,1.0,5.0,0,0.0,0.00,0,0.0,0.0,0.0
6,749579,2023-08,0,0.0,2400.00,1,1.0,1.0,-1.0,0,0.0,0.00,0,0.0,0.0,0.0



=== RESUMED TIMELINE (MOST RECENT RECORD) ===
Found 1 records in resumed_biopsy_pgta_timeline


,prontuario,period_month,biopsy_count,cumulative_biopsy_count,biopsy_payment_amount,biopsy_payment_count,biopsy_payment_qtd,cumulative_biopsy_payment_qtd,biopsy_missing_payment,pgta_test_count,cumulative_pgta_test_count,pgta_payment_amount,pgta_payment_count,pgta_payment_qtd,cumulative_pgta_payment_qtd,pgta_missing_payment
0,749579,2024-10,0,36.0,0.0,0,0.0,3.0,33.0,0,28.0,2369.09,1,2.0,16.0,12.0



QUERY SUMMARY FOR PRONTUARIO 749_579
- view_micromanipulacao: 4 records
- view_micromanipulacao_oocitos: 100 records
- view_tratamentos: 8 records
- view_embrioes_congelados: 36 records
- silver.mesclada_vendas: 84 records
- biopsy_pgta_timeline: 3 records
- billing_timeline: 6 records
- comprehensive_timeline: 7 records
- resumed_timeline: 1 records

Database connections closed.

Comprehensive Timeline for prontuario 749_579:
Total biopsies: 36
Total biopsy payments: 3
Biopsy missing payments: 33.0
Total PGT-A tests: 28
Total PGT-A payments: 4
PGT-A missing payments: 12.0


In [65]:
# Example: Query multiple prontuarios
# prontuarios = ['160751', '171805', '876950']
# all_results = {}
# 
# for pront in prontuarios:
#     print(f"\nQuerying prontuario: {pront}")
#     all_results[pront] = query_patient_data(pront)
# 
# # Compare results
# for pront, results in all_results.items():
#     print(f"Prontuario {pront}: {len(results['tratamentos'])} tratamentos")


In [66]:
df_mescladas =results_with_timeline['mesclada_vendas'].copy()

df_mescladas[df_mescladas['Descricao'].str.contains('PGT')]

,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
0,878497,LUCAS ROBERTO,11909,PATRICIA SOARES DOS SANTOS,111240,2025-09-29,ANALISE DE EMBRIOES PGT-A,6.0,6600.0,PGT-A,10150,502,35080,RPS,None,255,MATHEUS TEIXEIRA ROQUE,895430,,S3563.30,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100015,,,PGT-A,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,9990,20251002_152248,Mescladas2 (1).xlsx
3,878497,LUCAS ROBERTO,11909,PATRICIA SOARES DOS SANTOS,111240,2025-09-29,ANALISE DE EMBRIOES PGT-A,6.0,6600.0,PGT-A,10150,502,35083,RPS,34940,255,MATHEUS TEIXEIRA ROQUE,895430,,S3563.30,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100015,,,PGT-A,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,9997,20251002_152248,Mescladas2 (1).xlsx
